# Odata - check and restart pipelines

This notebook is run periodically from k972il's Jenkins server

In [14]:
import getpass, os

pipelines_url = os.environ.get('PIPELINES_URL') or getpass.getpass('PIPELINES_URL:')
force_restart = os.environ.get('FORCE_RESTART')

PIPELINES_URL: ······························································
FORCE_RESTART: 


In [15]:
import json, requests, subprocess, datetime

try:
  status = requests.get("{}/pipelines/api/raw/ckanext-upload_via_email/upload_via_email".format(pipelines_url)).json()
except Exception as e:
  print("Exception getting status: {}".format(e))
  status = {}
    
last_success = status.get('last_success')
print('last_success: {}'.format(datetime.datetime.fromtimestamp(last_success)))

def restart_pipelines():
  print("Restarting pipelines")
  cmd = 'kubectl -n odata-blue get pods | grep pipelines- | cut -f1 -d" "'
  status, output = subprocess.getstatusoutput(cmd)
  if status != 0: exit(1)
  for pod in output.split():
    print("Deleting pod {}".format(pod))
    print("status={} output={}".format(*subprocess.getstatusoutput("kubectl -n odata-blue delete pod {}".format(pod))))

if force_restart:
  print('forced restart')
  restart_pipelines()
if not last_success:
  print('could not find last success, restarting the pipelines')
  restart_pipelines()
elif (datetime.datetime.now() - datetime.datetime.fromtimestamp(last_success)).total_seconds()/60/60 > 2:
  print('last success is more then 2 hours ago, restarting the pipelines')
  restart_pipelines()

print('Great Success')

last_success: 2019-10-11 09:53:06.645970
Great Success
